By this notebook we are going to compare two different methods to solve Ising Model.

# Exact Diagonalization

In [ ]:
using LinearAlgebra 

$H = H_1 + H_2 =\sum_{<ij>} S^z_j S^z_{j+1} + \sum_j S^x_j $

In [ ]:
function Ising_exact_diag(N,h)
    # Introducing Components
    ⊗(x,y) = kron(x,y)
    id = [1 0;0 1]
    σˣ = [0 1;1 0]
    σᶻ = [1 0;0 -1]
    # H1, nearest neighbor interaction term
    H1 = fill(id,N)
    H1[1] = σᶻ
    H1[2] = σᶻ
    # H2, transverse field term
    H2 = fill(id,N)
    H2[1] = σˣ 
    # Creation of H
    H = zeros(Int, 2^N, 2^N)
    # Adding H1 to H
    for j in 1:N-1
        H -= foldl(⊗,H1)
        H1 = circshift(H1,1)
    end
    # Adding H2 to H
    for j in 1:N
        H -= h*foldl(⊗,H2)
        H2 = circshift(H2,1)
    end
    return H
end

In [ ]:
# test with sample N and h
H = Ising_exact_diag(5,1);

In [ ]:
# Compute eigenvalues and eigenstates of H
vals, vecs = eigen(H);

In [ ]:
# get the groundstate 
vals[1]

# DMRG method

In [ ]:
using ITensors 

In [ ]:
function Ising_dmrg(N,h)
    # Creation of sites
    sites = siteinds("S=1",N)
    # Make an OpSum object
    ops = OpSum()
    # First term of Hamiltonian
    for j=1:N-1
        ops -= "Sz",j,"Sz",j+1
    end 
    # Second term of Hamiltonian
    for j=1:N
        ops -= h,"Sx",j
    end
    # Construct MPO
    H_dmrg = MPO(ops,sites)
    # Construct MPS
    psi0 = randomMPS(sites,10)
    # Determine sweeps number
    swp = Sweeps(5)
    # Determine maximum dimension and cutoff
    setmaxdim!(swp,10,10,20,50,100)
    setcutoff!(swp,1E-10)
    return dmrg(H_dmrg,psi0,swp)
end 

In [ ]:
# Get energy and psi produced by dmrg algorithm
energy, psi = Ising_dmrg(5,1);